In [ ]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

# Load the CSV file with video paths and labels
df = pd.read_csv('video_gloss_mapping.csv')

# Base directory to save all extracted frames
save_base_dir = "extracted_frames_all"
os.makedirs(save_base_dir, exist_ok=True)

# Loop through each video entry in the CSV
for idx, row in df.iterrows():
    video_path = row['video_path']
    label = row['label']
    video_id = os.path.splitext(os.path.basename(video_path))[0]  # Extract video ID (e.g., '00335')

    if not os.path.exists(video_path):
        print(f"[{idx}] Skipped: Video file not found - {video_path}")
        continue

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"[{idx}] Skipped: Could not open video - {video_path}")
        continue

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count < 2:
        print(f"[{idx}] Skipped: Too few frames in - {video_path}")
        cap.release()
        continue

    num_frames_to_sample = min(10, frame_count)
    frame_indices = [int(frame_count * i / num_frames_to_sample) for i in range(1, num_frames_to_sample + 1)]

    # Create directory to save frames for this video under its label
    save_dir = os.path.join(save_base_dir, label, f"{video_id}")
    os.makedirs(save_dir, exist_ok=True)

    for i, frame_idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if not ret:
            print(f"Warning: Could not read frame {frame_idx} in video {video_path}")
            continue
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Save the frame
        frame_filename = os.path.join(save_dir, f"frame_{i}.jpg")
        cv2.imwrite(frame_filename, cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))

    cap.release()
    print(f"✅ Frame extraction completed for video: {os.path.basename(video_path)}")

# Final message after processing all videos
print("🎉 Frame extraction complete for all videos.")


✅ Frame extraction completed for video: 00335.mp4
✅ Frame extraction completed for video: 00336.mp4
✅ Frame extraction completed for video: 00338.mp4
✅ Frame extraction completed for video: 00339.mp4
✅ Frame extraction completed for video: 00341.mp4
✅ Frame extraction completed for video: 00376.mp4
✅ Frame extraction completed for video: 00377.mp4
✅ Frame extraction completed for video: 00381.mp4
✅ Frame extraction completed for video: 00382.mp4
✅ Frame extraction completed for video: 00384.mp4
✅ Frame extraction completed for video: 00414.mp4
✅ Frame extraction completed for video: 00415.mp4
✅ Frame extraction completed for video: 00416.mp4
✅ Frame extraction completed for video: 00421.mp4
✅ Frame extraction completed for video: 00426.mp4
✅ Frame extraction completed for video: 00430.mp4
✅ Frame extraction completed for video: 00431.mp4
✅ Frame extraction completed for video: 00433.mp4
✅ Frame extraction completed for video: 00435.mp4
✅ Frame extraction completed for video: 00583.mp4


In [5]:
import os
import cv2

# Input folder where extracted frames are currently stored
base_frames_dir = "extracted_frames_all"

# Output folder where resized frames will be saved
save_to = "resized_frames_all"
os.makedirs(save_to, exist_ok=True)

frame_counter = 0

for label in os.listdir(base_frames_dir):
    label_path = os.path.join(base_frames_dir, label)
    if not os.path.isdir(label_path):
        continue

    for video_id in os.listdir(label_path):
        video_path = os.path.join(label_path, video_id)
        if not os.path.isdir(video_path):
            continue

        resized_any = False

        for frame_file in os.listdir(video_path):
            frame_path = os.path.join(video_path, frame_file)
            try:
                img = cv2.imread(frame_path)
                if img is None:
                    print(f"❌ Could not read {frame_path}")
                    continue

                # Resize to 224x224
                img_resized = cv2.resize(img, (224, 224))

                # Prepare save path
                new_label_path = os.path.join(save_to, label, video_id)
                os.makedirs(new_label_path, exist_ok=True)
                new_frame_path = os.path.join(new_label_path, frame_file)

                # Save resized frame
                cv2.imwrite(new_frame_path, img_resized)
                frame_counter += 1
                resized_any = True

            except Exception as e:
                print(f"❌ Error processing {frame_path}: {e}")
                continue

        if resized_any:
            print(f"✅ Resized all frames of video: {video_id}.mp4")

print(f"\n🎉 All frames resized to 224x224. Total frames processed: {frame_counter}")


✅ Resized all frames of video: 01610.mp4
✅ Resized all frames of video: 01612.mp4
✅ Resized all frames of video: 01615.mp4
✅ Resized all frames of video: 66039.mp4
✅ Resized all frames of video: 02124.mp4
✅ Resized all frames of video: 02125.mp4
✅ Resized all frames of video: 02126.mp4
✅ Resized all frames of video: 02128.mp4
✅ Resized all frames of video: 02129.mp4
✅ Resized all frames of video: 02130.mp4
✅ Resized all frames of video: 02131.mp4
✅ Resized all frames of video: 00335.mp4
✅ Resized all frames of video: 00336.mp4
✅ Resized all frames of video: 00338.mp4
✅ Resized all frames of video: 00339.mp4
✅ Resized all frames of video: 00341.mp4
✅ Resized all frames of video: 00376.mp4
✅ Resized all frames of video: 00377.mp4
✅ Resized all frames of video: 00381.mp4
✅ Resized all frames of video: 00382.mp4
✅ Resized all frames of video: 00384.mp4
✅ Resized all frames of video: 00414.mp4
✅ Resized all frames of video: 00415.mp4
✅ Resized all frames of video: 00416.mp4
✅ Resized all fr

In [7]:
pip install torch torchvision numpy opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

# Paths
frames_root = "resized_frames_all"
features_root = "features_resnet"
os.makedirs(features_root, exist_ok=True)

# Load pretrained ResNet18 (remove final FC layer)
resnet = models.resnet18(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove final FC
resnet.eval()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)

# Preprocessing for ResNet
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Process each label
for label in os.listdir(frames_root):
    label_path = os.path.join(frames_root, label)
    if not os.path.isdir(label_path):
        continue

    for video_id in os.listdir(label_path):
        video_path = os.path.join(label_path, video_id)
        if not os.path.isdir(video_path):
            continue

        features = []

        for frame_file in sorted(os.listdir(video_path)):  # Sort to maintain order
            frame_path = os.path.join(video_path, frame_file)
            img = Image.open(frame_path).convert("RGB")
            img_tensor = preprocess(img).unsqueeze(0).to(device)

            with torch.no_grad():
                feature = resnet(img_tensor).squeeze().cpu().numpy()  # shape: (512,)
                features.append(feature)

        features = np.stack(features)  # shape: (num_frames, 512)

        # Save features
        save_dir = os.path.join(features_root, label)
        os.makedirs(save_dir, exist_ok=True)
        np.save(os.path.join(save_dir, f"{video_id}.npy"), features)

        print(f"✅ Feature extraction completed and saved for video: {video_id} ({label})")

print("🎉 All video frame features extracted and saved!")


C:\Users\deeks_w4ub1k8\Desktop\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\deeks_w4ub1k8\Desktop\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Feature extraction completed and saved for video: 01610 (a)
✅ Feature extraction completed and saved for video: 01612 (a)
✅ Feature extraction completed and saved for video: 01615 (a)
✅ Feature extraction completed and saved for video: 66039 (a)
✅ Feature extraction completed and saved for video: 02124 (a lot)
✅ Feature extraction completed and saved for video: 02125 (a lot)
✅ Feature extraction completed and saved for video: 02126 (a lot)
✅ Feature extraction completed and saved for video: 02128 (a lot)
✅ Feature extraction completed and saved for video: 02129 (a lot)
✅ Feature extraction completed and saved for video: 02130 (a lot)
✅ Feature extraction completed and saved for video: 02131 (a lot)
✅ Feature extraction completed and saved for video: 00335 (abdomen)
✅ Feature extraction completed and saved for video: 00336 (abdomen)
✅ Feature extraction completed and saved for video: 00338 (abdomen)
✅ Feature extraction completed and saved for video: 00339 (abdomen)
✅ Feature extracti

In [3]:
import numpy as np

data = np.load("C:\\Users\\deeks_w4ub1k8\\Sign2Speech\\features_resnet\\all\\01991.npy")
print("Shape:", data.shape)  # Should print something like (seq_len, 512)
print("First frame features:", data[0])


Shape: (9, 512)
First frame features: [8.28062475e-01 1.36654651e+00 0.00000000e+00 3.89810979e-01
 3.04080188e-01 9.91399169e-01 3.68302991e-03 1.75284743e-01
 1.94456565e+00 1.44359574e-01 3.15575957e-01 1.09936714e-01
 5.96307814e-01 3.27974558e-01 1.35274515e-01 6.36793256e-01
 0.00000000e+00 4.70405102e-01 3.91201563e-02 2.07868266e+00
 8.75784397e-01 1.19869328e+00 5.34375072e-01 5.94992638e-01
 1.09551616e-01 2.19645560e-01 6.61598206e-01 3.83226633e+00
 2.05531865e-01 1.05544269e+00 6.03497207e-01 1.53639361e-01
 5.00194490e-01 8.83928776e-01 1.60521376e+00 1.20851927e-01
 1.16481102e+00 1.80684015e-01 4.81216580e-01 6.52287379e-02
 2.53888905e-01 5.84860027e-01 6.84846938e-02 8.82207692e-01
 9.17941332e-01 9.12735146e-03 2.31088281e-01 2.11771384e-01
 2.87752897e-01 2.13390994e+00 1.48206365e+00 4.92084622e-02
 2.64573902e-01 4.90316182e-01 2.53179288e+00 1.66242898e+00
 5.98536432e-01 1.19850624e+00 2.19296575e-01 1.65947425e+00
 2.80200988e-01 7.96406388e-01 6.13238215e-01 4